Introduction:
-------------------

This Notebook contains some codes to generate simulated data to mimic polarizers like IAGPOL polarimeter. It is being used to test `astropop` reduction code against `pccdpack`, the standard code for that instrument.

To run the tests you must have `IRAF` and `pccdpack` installed. `pccdpack` is not OpenSource, so you should ask for the code. If you simply want to simulate, only `astropop` is needed.

This code will be better documented soon.


References:
------------------
1 - IAGPOL homepage: http://astroweb.iag.usp.br/~polarimetria/gaveta/default.htm

2 - astropop repository: https://github.com/juliotux/astropop

In [ ]:
from astropy.io import fits
from astropy.table import vstack, Table
from astropop.math.models.moffat import moffat_2d
from astropop.math.array import trim_array
from astropop.py_utils import mkdir_p, check_iterable
from astropop.pipelines.polarimetry_scripts import run_pccdpack, process_polarimetry
from astropop.photometry import process_photometry
import numpy as np
from matplotlib import pyplot as plt
%matplotlib notebook

from astropop.logger import logger
np.warnings.filterwarnings('ignore')
logger.setLevel('INFO')

This function `gen_image_pol` generate a single image of a polarization dataset from a instrument like IAGPOL. 

In [ ]:
def gen_image_pol(x, y, psi, p, theta, shape, fwhm):
    """Single image generation
    
    x, y : arrays -> star coordinates
    psi : float -> angle of the half-wave retarder
    p : float or array -> polarization level (ratio, not percent)
    theta : float or array -> position angle of polarization
    shape : (y, x) -> image size
    fwhm : float - FWHM of the stellar PSF.
    """
    rdnoise = 18.5   # Readnoise of the camera
    sky = 300        # Sky value
    slope = 1.0      # Scale of the exponential distribution of the stars
    max_flux=700000  # Maximum allowed flus
    xs = np.array(x)
    ys = np.array(y)
    n = len(xs)
    beta = 1.5       # beta of Moffat distribution
    alpha = fwhm/2*np.sqrt(2**(1/beta) - 1)
    threshold = 10**-5 # minimum value for bounding box the stars.
    
    if not check_iterable(p):
        # this ensures q, u, z, c are arrays
        p = np.array([p]*len(xs))

    q = p*np.cos(2*np.radians(theta))
    u = p*np.sin(2*np.radians(theta))
    
    z = q*np.cos(4*np.radians(psi)) + u*np.sin(4*np.radians(psi))
    c = (z+1)/2
    
    base_im = np.zeros(shape)

    fluxes = np.random.exponential(1.0, n)
    fluxes = np.clip(fluxes, 0, 5)*max_flux/5
    
    if np.max([c, 1-c])*np.max(fluxes)*(beta-1)/(np.pi*alpha**2) > 65000:
        logger.warn("Probably you have saturated stars in this field.")
    
    indices = np.indices(shape)
    for xi, yi, sca, rat in zip(xs, ys, fluxes, c):
        bbox = int(np.sqrt((threshold**(-1/beta) - 1)*alpha))
        for diff, rel in zip((0, 25), (rat, 1-rat)):
            xii, yii = xi-diff, yi+diff
            _, ix, iy = trim_array(base_im, bbox, (xii, yii), indices)
            im = moffat_2d(ix, iy, xii, yii, alpha, beta, sca, 0)*rel
            im = np.random.poisson(im)
            im = np.round(im)
            base_im[iy, ix] += im
    
    base_im += np.random.normal(sky, rdnoise, shape).astype('uint16')            
    cat = Table([xs, ys, xs-25, ys+25, fluxes], names=('xo', 'yo', 'xe', 'ye', 'fluxes'))
    
    return base_im, cat

# Simulating the Fields

### Generate a single dataset with 200 stars with random polarization and theta.

In [ ]:
n = 200
p = np.random.random(n)*0.3  # Random in range [0, 0.3)
theta = np.random.random(n)*180 # random in range [0, 180)
shape = (2048, 2048)

laminas = [0, 1, 2, 3,
           4, 5, 6, 7,
           8, 9, 10, 11,
           12, 13, 14, 15]

x = np.random.random(n)*shape[1]
y = np.random.random(n)*shape[0]
images = []
for i in laminas:
    # Create one image for each retarder position (lamina)
    # and create a header like IAGPOL
    ik = hex(i)[2]
    im, cat = gen_image_pol(x, y, i*22.5, p, theta, shape, fwhm=5)
    head = fits.Header()
    head['LAMINA'] = f'L{ik}'
    head['LAM-POS'] = f'{i}'
    head['FILTER'] = 'F2'
    head['FIL-POS'] = '2'
    head['GAIN'] = 1.0
    head['RDNOISE'] = 18.5
    hdu = fits.PrimaryHDU(im.astype('uint16'), header=head)
    images.append(hdu)

k = run_pccdpack(images, retarder_type='half', retarder_direction=1,
                 retarder_rotation=22.5, retarder_key='LAM-POS',
                 rdnoise_key='RDNOISE', astrometry_calib=False,
                 r=np.arange(2, 18, 1), r_in=50, r_out=60,
                 detect_fwhm=5, detect_snr=5)
a = process_polarimetry(images, align_images=False, retarder_type='half',
                        retarder_direction=1,retarder_rotation=22.5, retarder_key='LAM-POS',
                        rdnoise_key='RDNOISE', astrometry_calib=False,
                        r=np.arange(2, 18, 1), r_in=50, r_out=60,
                        detect_fwhm=5, detect_snr=5, photometry_type='aperture',
                        delta_x=-25, delta_y=25, calculate_mode='fit')
b = process_polarimetry(images, align_images=False, retarder_type='half',
                        retarder_direction=1,retarder_rotation=22.5, retarder_key='LAM-POS',
                        rdnoise_key='RDNOISE', astrometry_calib=False,
                        r=np.arange(2, 18, 1), r_in=50, r_out=60,
                        detect_fwhm=5, detect_snr=5, photometry_type='aperture',
                        delta_x=-25, delta_y=25, calculate_mode='sum')

pccd_out = k[0]
pccd_log = k[2]
apop_fit = a[0]
apop_mbr = b[0]

pccd_out.write('pol_models/prandom.pccd_out.fits')
pccd_log.write('pol_models/prandom.pccd_log.fits')
apop_fit.write('pol_models/prandom.apop_fit.fits')
apop_mbr.write('pol_models/prandom.apop_mbr.fits')

### Generate 5 datasets with fixed polarization (P=10%) and theta (15°).

In [ ]:
p = 0.1              # Polarization of all stars
theta = 15           # Theta of all stars
shape = (2048, 2048) # Image shape
n = 200              # Number of stars

results = {'pccd_log': Table(),  # PCCDPACK log files
           'pccd_out': Table(),  # PCCDPACK out files
           'mbr84': Table(),     # ASTROPOP polarimetry by MBR84 algorithm
           'fit': Table(),       # ASTROPOP polarimetry by fitting
          }

laminas = [0, 1, 2, 3,
           4, 5, 6, 7,
           8, 9, 10, 11,
           12, 13, 14, 15]

for f in range(5):
    images = []
    x = np.random.random(n)*shape[1]
    y = np.random.random(n)*shape[0]
    for i in laminas:
        # Create one image for each retarder position (lamina)
        # and create a header like IAGPOL
        ik = hex(i)[2]
        im, cat = gen_image_pol(x, y, i*22.5, p, theta, shape, fwhm=5)
        head = fits.Header()
        head['LAMINA'] = f'L{ik}'
        head['LAM-POS'] = f'{i}'
        head['FILTER'] = 'F2'
        head['FIL-POS'] = '2'
        head['GAIN'] = 1.0
        head['RDNOISE'] = 18.5
        hdu = fits.PrimaryHDU(im.astype('uint16'), header=head)
        images.append(hdu)
    try:
        # Compute the polarimetry by ASTROPOP and PCCPACK
        k = run_pccdpack(images, retarder_type='half', retarder_direction=1,
                         retarder_rotation=22.5, retarder_key='LAM-POS',
                         rdnoise_key='RDNOISE', astrometry_calib=False,
                         r=np.arange(2, 18, 1), r_in=50, r_out=60,
                         detect_fwhm=5, detect_snr=5)
        a = process_polarimetry(images, align_images=False, retarder_type='half',
                                retarder_direction=1,retarder_rotation=22.5, retarder_key='LAM-POS',
                                rdnoise_key='RDNOISE', astrometry_calib=False,
                                r=np.arange(2, 18, 1), r_in=50, r_out=60,
                                detect_fwhm=5, detect_snr=5, photometry_type='aperture',
                                delta_x=-25, delta_y=25, calculate_mode='fit')
        b = process_polarimetry(images, align_images=False, retarder_type='half',
                                retarder_direction=1,retarder_rotation=22.5, retarder_key='LAM-POS',
                                rdnoise_key='RDNOISE', astrometry_calib=False,
                                r=np.arange(2, 18, 1), r_in=50, r_out=60,
                                detect_fwhm=5, detect_snr=5, photometry_type='aperture',
                                delta_x=-25, delta_y=25, calculate_mode='sum')
        a[0]['set'] = [f]*len(a[0])
        b[0]['set'] = [f]*len(b[0])
        k[0]['set'] = [f]*len(k[0])
        k[2]['set'] = [f]*len(k[2])
        results['fit'] = vstack([results['fit'], a[0]])
        results['mbr84'] = vstack([results['mbr84'], b[0]])
        results['pccd_log'] = vstack([results['pccd_log'], k[2]])
        results['pccd_out'] = vstack([results['pccd_out'], k[0]])
    except:
        raise

results['fit'].write('pol_models/5sets.fit.fits')
results['mbr84'].write('pol_models/5sets.hbr84.fits')
results['pccd_out'].write('pol_models/5sets.pccd_out.fits')
results['pccd_log'].write('pol_models/5sets.pccd_log.fits')